In [32]:
# Setting up the environment

import pandas as pd
from dotenv import load_dotenv
import os 

load_dotenv()

df: pd.Series = pd.read_json('db.json',typ='series')
df = df[:5]

from utils.html_tools import extract_text

df = df.apply(lambda x: extract_text(x))
print(df)
from utils.open_ai_tools import parse_to_blink
import openai
# OpenAI Integration
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "OOPS, please set env var called 'OPEN_AI_API_KEY'")
from langchain.llms import OpenAI

index                     Angular Documentation Angular is an applicati...
guide/what-is-angular     What is Angular? This topic can help you unde...
start/start-routing       Adding navigation This guide builds on the fi...
start/start-forms         Using forms for user input This guide builds ...
guide/setup-local         Setting up the local environment and workspac...
dtype: object


In [33]:
from typing import Optional, List
from pydantic import BaseModel, Field
from enum import Enum

class Difficulty(str, Enum):
    beginner = "beginner"
    intermediate = "intermediate"
    advanced = "advanced"

class Blink(BaseModel):
    title: str = Field(..., example="Object Oriented Programming")
    description: str = Field(..., example="A programming paradigm based on the concept of objects, which can contain data and code: data in the form of fields, and code, in the form of procedures.")
    code: Optional[str] = Field(..., example="class Person:\n    def __init__(self, name, age):\n        self.name = name\n        self.age = age\n\np1 = Person(\"John\", 36)\nprint(p1.name)\nprint(p1.age)")
    difficulty: Difficulty = Difficulty.beginner

class Blinks(BaseModel):
    blinks: List[Blink] = []


In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import json

from utils.open_ai_tools import truncate_text

blinks = {}
for index, i in enumerate(df.values):
    query = HumanMessage(content=i)
    llm = ChatOpenAI()
    functions = [
        {
            "name": "get_blinks",
            "description": "Extract blinks from a html page",
            "parameters": Blinks.schema()
        }
    ]
    
    template = """
    You are a programming teacher for beginners. You talk only in json and you dont use newline characters outside of json values.
    Generate a json response that teaches something and explains a concept from the provided HTML content (make the lesson universal, IMPORTANT! This is used in a 3rd party app, dont make any references to the original docs), I will kill myself if i see anything else. adhering to this JSON structure: 
    The code can be left empty if it doesnt make sense to have code in the lesson.
    Use emojis to make the lesson more fun!
    The function get_blinks() should return a list of blinks, each blink should have the following structure:
    (
    "title": "<title>",
    "description": "<explanation>", # Make this sound human
    "code": "<code>"
    "difficulty": "<beginner | intermediate | advanced>"
    )

    HTML content to convert into a blink, each page can have multiple blinks!: {html}
    """

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": template.format(html=query.content)},
        ],
        functions=functions,
        function_call="auto",
    )
    blinks[index] = response.choices[0].message.function_call.arguments



In [35]:
blinks

{0: '{\n  "blinks": [\n    {\n      "title": "Introduction to Angular 🚀",\n      "description": "Angular is a robust framework for building efficient and complex single-page web apps. It\'s great for both beginners working on their first app, and professionals who are optimizing elaborate apps for companies. Handy feature: it comes with tutorials and guides, complete with downloadable examples for you to kick-start your projects! 😄",\n      "code": "",\n      "difficulty": "beginner"\n    },\n    {\n      "title": "Assumed Knowledge 🧠",\n      "description": "Hold up! Before diving into Angular, you should already have a basic understanding of HTML, CSS, JavaScript, and latest standard tools like classes and modules. While the samples they provide are in TypeScript, it\'s important to know that most Angular code can be written with just the latest JavaScript. So, it\'s not necessary to be a TypeScript pro to get started! 🤓",\n      "code": "",\n      "difficulty": "beginner"\n    },\n 

In [74]:
# Write to file
parsed_blinks = {'blinks': []}
for blink_set in blinks:
    loaded_json = json.loads(blinks[blink_set])
    for blink in loaded_json['blinks']:
        parsed_blinks['blinks'].append(blink)
    
print(parsed_blinks)
with open('blinks.json', 'w') as f:
        f.write(json.dumps(parsed_blinks))

{'blinks': [{'title': 'Introduction to Angular 🚀', 'description': "Angular is a robust framework for building efficient and complex single-page web apps. It's great for both beginners working on their first app, and professionals who are optimizing elaborate apps for companies. Handy feature: it comes with tutorials and guides, complete with downloadable examples for you to kick-start your projects! 😄", 'code': '', 'difficulty': 'beginner'}, {'title': 'Assumed Knowledge 🧠', 'description': "Hold up! Before diving into Angular, you should already have a basic understanding of HTML, CSS, JavaScript, and latest standard tools like classes and modules. While the samples they provide are in TypeScript, it's important to know that most Angular code can be written with just the latest JavaScript. So, it's not necessary to be a TypeScript pro to get started! 🤓", 'code': '', 'difficulty': 'beginner'}, {'title': 'The Power of TypeScript in Angular 💪', 'description': "While you can use just JavaSc

In [76]:
with open('blinks.json', 'r') as f:
    for line in f:
        loaded_blink = json.loads(line)
        loaded_blinks = loaded_blink['blinks']
    
loaded_blinks

[{'title': 'Introduction to Angular 🚀',
  'description': "Angular is a robust framework for building efficient and complex single-page web apps. It's great for both beginners working on their first app, and professionals who are optimizing elaborate apps for companies. Handy feature: it comes with tutorials and guides, complete with downloadable examples for you to kick-start your projects! 😄",
  'code': '',
  'difficulty': 'beginner'},
 {'title': 'Assumed Knowledge 🧠',
  'description': "Hold up! Before diving into Angular, you should already have a basic understanding of HTML, CSS, JavaScript, and latest standard tools like classes and modules. While the samples they provide are in TypeScript, it's important to know that most Angular code can be written with just the latest JavaScript. So, it's not necessary to be a TypeScript pro to get started! 🤓",
  'code': '',
  'difficulty': 'beginner'},
 {'title': 'The Power of TypeScript in Angular 💪',
  'description': "While you can use just J